In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
# df = pd.read_csv("data/TXF1-Tick-Trade.txt", parse_dates = {"date": ["Date", "Time"]})
df = pd.read_csv("data/TXF1-KWAY64V2-TAIFEX-Futures-Minute-Trade.txt", parse_dates = {"date": ["Date", "Time"]})
print(df.shape)
df.head()

(2069610, 7)


,date,Symbol,Open,High,Low,Close,TotalVolume
0,2010-01-04 08:46:00,TXF1,8203.0,8208.0,8184.0,8189.0,1125
1,2010-01-04 08:47:00,TXF1,8188.0,8195.0,8186.0,8193.0,411
2,2010-01-04 08:48:00,TXF1,8194.0,8201.0,8193.0,8201.0,422
3,2010-01-04 08:49:00,TXF1,8201.0,8203.0,8198.0,8201.0,297
4,2010-01-04 08:50:00,TXF1,8199.0,8200.0,8195.0,8197.0,189


In [3]:
df["time"] = df["date"].dt.time
df["time"] = df["time"].astype(str).str.replace(":", "")
df["day_night"] = df.eval("(time >= '084500') & (time < '134500')")
df = df[df["day_night"]].reset_index(drop = True)

# df["Price"] = df["Price"].astype(int)

df = df.drop(["Symbol", "time", "day_night"], axis = 1)
df.columns = ["date", "open", "high", "low", "close", "volume"]

print(df.shape)
df.head()

(953624, 6)


,date,open,high,low,close,volume
0,2010-01-04 08:46:00,8203.0,8208.0,8184.0,8189.0,1125
1,2010-01-04 08:47:00,8188.0,8195.0,8186.0,8193.0,411
2,2010-01-04 08:48:00,8194.0,8201.0,8193.0,8201.0,422
3,2010-01-04 08:49:00,8201.0,8203.0,8198.0,8201.0,297
4,2010-01-04 08:50:00,8199.0,8200.0,8195.0,8197.0,189


df = df.groupby("date").agg({"Price": ["first", "max", "min", "last"], "Volume": "sum"})
df.columns = ["open", "high", "low", "close", "volume"]
df = df.reset_index()
df = df.sort_values("date").reset_index(drop = True)
#df.to_excel("data/TICKS_day_min.xlsx", index = False)
df

#### 策略
該秒成交量超過100口的情況，接下來"十秒內"  
跌破該秒低點->做空  
超過該秒高點->做多  
同時突破 -> 不做  

* 一次只做一口 (Done)
* 若已經持倉就不做任何動作 (Done)
* 不留倉->收盤價清倉 (Done)
* 如果手上有多/空單，但觸發空/多單訊號的->收盤價平倉 (Done)

測試:  
停利60、停損20  
090500 - 132459

In [4]:
volume = 1500
df[f"vol_{volume}"] = (df["volume"] > volume).astype(int)
print(df.shape)
df.head()

(953624, 7)


,date,open,high,low,close,volume,vol_1500
0,2010-01-04 08:46:00,8203.0,8208.0,8184.0,8189.0,1125,0
1,2010-01-04 08:47:00,8188.0,8195.0,8186.0,8193.0,411,0
2,2010-01-04 08:48:00,8194.0,8201.0,8193.0,8201.0,422,0
3,2010-01-04 08:49:00,8201.0,8203.0,8198.0,8201.0,297,0
4,2010-01-04 08:50:00,8199.0,8200.0,8195.0,8197.0,189,0


In [5]:
period = 10
for i in tqdm(range(len(df))):
    if df.loc[i, f"vol_{volume}"] == 1:
        df.loc[i+1:i+period, "prev_time"] =  df.loc[i, "date"]
        df.loc[i+1:i+period, "prev_high"] = df.loc[i, "high"]
        df.loc[i+1:i+period, "prev_low"] = df.loc[i, "low"]

100%|██████████| 953624/953624 [00:12<00:00, 75583.07it/s] 


In [6]:
df["break_high"] = df.eval("high > prev_high").astype(int)
df["break_low"] = df.eval("low < prev_low").astype(int)

In [7]:
for i in tqdm(range(len(df))):
    if (df.loc[i, "break_high"] == 1) & (df.loc[i, "break_low"] == 0):
        df.loc[i, "direction"] = "long"
        
    elif  (df.loc[i, "break_high"] == 0) & (df.loc[i, "break_low"] == 1):
        df.loc[i, "direction"] = "short"

100%|██████████| 953624/953624 [00:26<00:00, 35856.19it/s]


In [8]:
df.insert(1, "time", df["date"].dt.time.astype(str).str.replace(":", ""))

In [9]:
hold = 0
slippage = 2
fee = 2
profit_limit = 60
loss_limit = 20
start_time = "090500" # "084500" "090500"
end_time = "132459" # "134459" "132459"

adjust_cost = slippage + fee
df["hold_time"] = None
df[["hold", "long", "short", "cost", "profit", "loss", "offset", "clear", "win", "net"]] = 0

for i in tqdm(range(1, len(df))):
    if (df.loc[i, "time"] >= start_time) & (df.loc[i, "time"] <= end_time):
        # 持有損益
        if (hold == 1):
            if df.loc[i-1, "long"] == 1:
                df.loc[i, "hold_time"] = df.loc[i-1, "hold_time"]
                df.loc[i, "hold"] = hold
                df.loc[i, "long"] = 1
                df.loc[i, "cost"] = df.loc[i-1, "cost"]

                df.loc[i, "profit"] = max(df.loc[i, "high"] - df.loc[i, "cost"], 0)
                df.loc[i, "loss"] = max(df.loc[i, "cost"] - df.loc[i, "low"], 0)

            elif df.loc[i-1, "short"] == 1:
                df.loc[i, "hold_time"] = df.loc[i-1, "hold_time"]
                df.loc[i, "hold"] = hold
                df.loc[i, "short"] = 1
                df.loc[i, "cost"] = df.loc[i-1, "cost"]

                df.loc[i, "profit"] = max(df.loc[i, "cost"] - df.loc[i, "low"], 0)
                df.loc[i, "loss"] = max(df.loc[i, "high"] - df.loc[i, "cost"], 0)
        # 觸發操作
        else:         
            if df.loc[i, "direction"] == "long":
                hold = 1
                df.loc[i, "hold_time"] = df.loc[i, "date"]
                df.loc[i, "hold"] = hold
                df.loc[i, "long"] = 1
                df.loc[i, "cost"] = df.loc[i, "high"]
                
                df.loc[i, "profit"] = max(df.loc[i, "high"] - df.loc[i, "cost"], 0)
                df.loc[i, "loss"] = max(df.loc[i, "cost"] - df.loc[i, "low"], 0)
                
            elif df.loc[i, "direction"] == "short":
                hold = 1
                df.loc[i, "hold_time"] = df.loc[i, "date"]
                df.loc[i, "hold"] = hold
                df.loc[i, "short"] = 1
                df.loc[i, "cost"] = df.loc[i, "low"]
                
                df.loc[i, "profit"] = max(df.loc[i, "cost"] - df.loc[i, "low"], 0)
                df.loc[i, "loss"] = max(df.loc[i, "high"] - df.loc[i, "cost"], 0)
        

        if  (hold == 1):
            if (df.loc[i, "profit"] >= profit_limit):# 停利
                df.loc[i, "win"] = 1
                df.loc[i, "net"] = profit_limit
                hold = 0
                
            elif (df.loc[i, "loss"] >= loss_limit): # 停損
                df.loc[i, "win"] = -1
                df.loc[i, "net"] = -loss_limit
                hold = 0

            elif (df.loc[i, "long"] == 1) & (df.loc[i, "direction"] == "short"): # 多單平倉
                df.loc[i, "offset"] = 1
                df.loc[i, "net"] = df.loc[i, "close"] - df.loc[i, "cost"]
                df.loc[i, "win"] = 1 if df.loc[i, "net"] > 0 else -1
                hold = 0
            elif (df.loc[i, "short"] == 1) & (df.loc[i, "direction"] == "long"): # 空單平倉
                df.loc[i, "offset"] = 1
                df.loc[i, "net"] = df.loc[i, "cost"] - df.loc[i, "close"]
                df.loc[i, "win"] = 1 if df.loc[i, "net"] > 0 else -1
                hold = 0

    else:
        # 清倉
        if (hold == 1):
            df.loc[i-1, "clear"] = 1
            if df.loc[i-1, "long"] == 1:
                df.loc[i-1, "net"] = df.loc[i-1, "close"] - df.loc[i-1, "cost"]
            elif df.loc[i-1, "short"] == 1:
                df.loc[i-1, "net"] = df.loc[i-1, "cost"] - df.loc[i-1, "close"]

            df.loc[i-1, "win"] = 1 if df.loc[i-1, "net"] > 0 else -1
            
            hold = 0
            
df60_20_090500_132459 = df.copy()

100%|██████████| 953623/953623 [04:37<00:00, 3435.06it/s] 


In [10]:
dfs = [
    [df60_20_090500_132459, "停利60、停損20、090500-132459"],
    ]

In [11]:
counts = pd.DataFrame()
for df, name in dfs:
    count = pd.DataFrame(df["win"].value_counts().loc[[1, -1]]).T
    count.index = [name]
    count.columns = ["win", "lose"]
    count["total"] =  count.sum(axis = 1)
    count["total_net"] = df["net"].sum()
    counts = pd.concat([counts, count])
counts

,win,lose,total,total_net
停利60、停損20、090500-132459,3372,11593,14965,-113894


In [12]:
with pd.ExcelWriter("data/strategy1.xlsx") as writer:
    counts.to_excel(writer, sheet_name = "統計")
    for df, name in dfs:
        df.to_excel(writer, sheet_name = name)

show_col = [
    'date', 'open', 'high', 'low', 'close', 'volume', 'direction', 'hold_time', 'hold', 'long', 'short', 'cost', 'profit',
    'loss', 'offset', 'clear', 'win', 'net'
    ]

df.query("(time > '132459') & (hold == 1)")[show_col]

df[show_col].query("win == 1")

df[show_col].query("win == -1")

df[show_col].query("clear == 1")

df[show_col].query("offset == 1")